# 🦌 ELK Transformer 🤖

A transformer object that will convert some input source into valid Elk Json. This
example using a transformer around a networkx graph.

In [ ]:
if __name__ == "__main__":
    %pip install -q -r requirements.txt

In [ ]:
import json
import pathlib

import ipywidgets
import networkx
from IPython.display import display

import ipyelk
from ipyelk.tools import ToggleCollapsedTool

NX_VINFO = tuple(map(int, networkx.__version__.split(".")[:2]))
NX_EDGES = "edges" if NX_VINFO >= (3, 4) else "link"

In [ ]:
def load_nx_graph(filename: str) -> networkx.MultiDiGraph:
    return networkx.readwrite.json_graph.node_link_graph(
        json.loads(pathlib.Path(filename).resolve().read_text(encoding="utf-8")),
        **{NX_EDGES: "links"},
    )

## Flat structure

A `networkx.MultiDigraph` can be used to create a flat graph.

> _TODO: There should be an option to specify if ports should be created or only connect
> edges between the nodes_

In [ ]:
def a_flat_elk_json_example(graph: networkx.MultiDiGraph = None):
    graph = graph or load_nx_graph("flat_graph.json")
    elk = ipyelk.from_nx(graph=graph, layout=dict(min_height="200px"))
    return elk

In [ ]:
if __name__ == "__main__":
    flat = a_flat_elk_json_example()
    display(flat)

## Hierarchical Diagram with Ports

In [ ]:
def a_hierarchical_elk_example(
    tree: networkx.MultiDiGraph = None, ports: networkx.MultiDiGraph = None
):
    tree = tree or load_nx_graph("hier_tree.json")
    ports = ports or load_nx_graph("hier_ports.json")
    elk = ipyelk.from_nx(
        graph=ports,
        hierarchy=tree,
        layout=dict(
            min_height="200px",
            height="100%",
        ),
    )
    return elk

In [ ]:
def a_collapsible_elk_example(elk=None):
    elk = elk or a_hierarchical_elk_example()

    collapser = elk.get_tool(ToggleCollapsedTool)
    toggle = ipywidgets.Button(description="Toggle")
    toggle.on_click(collapser.handler)

    box = ipywidgets.VBox(
        [
            ipywidgets.HBox([
                ipywidgets.HTML("<h2>👇 click a group node then click 👉</h2>"),
                toggle,
            ]),
            elk,
        ],
        layout={
            "height": "100%",
        },
    )
    return box, elk

In [ ]:
if __name__ == "__main__":
    hier_box, hier_elk = a_collapsible_elk_example()
    display(hier_box)

## 🦌 Learn More 📖

See the [other examples](./_index.ipynb).